# Semi Gradient n-step SARSA
> Mountain Car Problem


In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from gymnasium.wrappers import RecordVideo
import datetime

## Env

https://gymnasium.farama.org/environments/classic_control/mountain_car/


In [ ]:
env = gym.make("MountainCar-v0", render_mode="rgb_array")

# increase episode length from 200 to 4000
env._max_episode_steps = 4000

np.random.seed(13)
env.reset()
plt.imshow(env.render())

In [ ]:
state, _ = env.reset()
print(state)

In [ ]:
print(env.action_space.sample())

## Random Action


In [ ]:
total_reward = 0.0
total_steps = 0
timestamp = int(datetime.datetime.now().timestamp())
env = RecordVideo(env=env, video_folder="./video", name_prefix=timestamp)
obs, _ = env.reset()

done = False
while not done:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    total_steps += 1
    done = terminated or truncated
    if total_steps % 200 == 0:
        print(
            f"Step {total_steps:2d}: action={action:2d}, Reward={reward:5.2f}, "
            f"Terminated={terminated}, Truncated={truncated}, info={info}"
        )

print(
    f"Episode finished after {total_steps} steps with total reward: {total_reward:.2f}"
)
env.close()

## Reinforcement Learning


In [ ]:
# The implementation of tiles3 is from Richard Sutton's website
# http://incompleteideas.net/tiles/tiles3.html
from tiles3 import IHT, tiles

In [ ]:
class QEstimator:
    def __init__(
        self, step_size, num_of_tilings=8, tiles_per_dim=8, max_size=2048, epsilon=0.0
    ):
        self.max_size = max_size
        self.num_of_tilings = num_of_tilings
        self.tiles_per_dim = tiles_per_dim
        self.epsilon = epsilon
        self.step_size = step_size / num_of_tilings

        self.table = IHT(max_size)

        self.w = np.zeros(max_size)

        self.pos_scale = self.tiles_per_dim / (
            env.observation_space.high[0] - env.observation_space.low[0]
        )
        self.vel_scale = self.tiles_per_dim / (
            env.observation_space.high[1] - env.observation_space.low[1]
        )

    def get_active_features(self, state, action):
        pos, vel = state
        active_features = tiles(
            self.table,
            self.num_of_tilings,
            [
                self.pos_scale * (pos - env.observation_space.low[0]),
                self.vel_scale * (vel - env.observation_space.low[1]),
            ],
            [action],
        )
        return active_features

    def q_predict(self, state, action):
        pos, vel = state
        if pos == env.observation_space.high[0]:  # reached goal
            return 0.0
        else:
            active_features = self.get_active_features(state, action)
            return np.sum(self.w[active_features])

    # learn with given state, action and target
    def q_update(self, state, action, target):
        active_features = self.get_active_features(state, action)
        q_s_a = np.sum(self.w[active_features])
        delta = target - q_s_a
        self.w[active_features] += self.step_size * delta

    def get_eps_greedy_action(self, state):
        pos, vel = state
        if np.random.rand() < self.epsilon:
            return np.random.choice(env.action_space.n)
        else:
            qvals = np.array(
                [self.q_predict(state, action) for action in range(env.action_space.n)]
            )
            return np.argmax(qvals)

In [ ]:
def sarsa_n(qhat, env, n=1, gamma=1.0, episode_cnt=10000):
    episode_rewards = []
    print_every = max(1, episode_cnt // 10)
    for ec in range(episode_cnt):
        if ec % print_every == 0 or ec == episode_cnt - 1:
            print(f"Episode: {ec + 1}/{episode_cnt}")
        state, _ = env.reset()
        action = qhat.get_eps_greedy_action(state)
        T = float("inf")
        t = 0
        states = [state]
        actions = [action]
        rewards = [0.0]
        while True:
            if t < T:
                next_state, reward, done, _, _ = env.step(action)
                states.append(next_state)
                rewards.append(reward)

                if done:
                    T = t + 1
                else:
                    next_action = qhat.get_eps_greedy_action(next_state)
                    actions.append(next_action)

            tau = t - n + 1

            if tau >= 0:
                G = 0
                for i in range(tau + 1, min(tau + n, T) + 1):
                    G += gamma ** (i - tau - 1) * rewards[i]
                if tau + n < T:
                    G += gamma**n * qhat.q_predict(states[tau + n], actions[tau + n])
                qhat.q_update(states[tau], actions[tau], G)

            if tau == T - 1:
                episode_rewards.append(np.sum(rewards))
                break
            else:
                t += 1
                state = next_state
                action = next_action

    return np.array(episode_rewards)

In [ ]:
# plot rewards
def plot_rewards(env_name, rewards, label):
    plt.title("env={}, Mean reward = {:.1f}".format(env_name, np.mean(rewards[-20:])))
    plt.plot(rewards, label=label)
    plt.grid()
    plt.legend()
    plt.ylim(-500, 0)
    plt.show()

In [ ]:
# Initialize environment
env = gym.make("MountainCar-v0", render_mode="rgb_array")
state, _ = env.reset(seed=13)
print(state)

# create a n-SARSA Learning agent
step_size = 0.8
episode_cnt = 1000
n = 4
epsilon = 0.01
gamma = 1.0
estimator = QEstimator(step_size, epsilon=epsilon)
reward_arr = []

In [ ]:
episode_cnt = 100

rewards = sarsa_n(
    estimator,
    env=env,
    n=n,
    gamma=gamma,
    episode_cnt=episode_cnt,
)
reward_arr = [*reward_arr, *rewards]

# plot rewards
plot_rewards("Mountain Car World", reward_arr, "Semi Grad n-step SARSA")

### Take a look into the agent


In [ ]:
state = env.observation_space.sample()
action = env.action_space.sample()
print("=" * 40)
print(f"Sampled state:  {state}")
print(f"Sampled action: {action}")
print("=" * 40)

active_features = estimator.get_active_features(state=state, action=action)
print("Active features for selected state-action pair:")
print(active_features)
print("=" * 40)

q_pred = [estimator.q_predict(state=state, action=a) for a in range(env.action_space.n)]
print("Q-value predictions for all actions:")
for idx, q in enumerate(q_pred):
    print(f"  Action {idx}: {q:.3f}")
print("=" * 40)

a_greedy = estimator.get_eps_greedy_action(state=state)
print(f"Epsilon-greedy action for sampled state: {a_greedy}")
print("=" * 40)

In [ ]:
env.close()

## Take a look at the result


In [ ]:
env = gym.make("MountainCar-v0", render_mode="rgb_array")
timestamp = int(datetime.datetime.now().timestamp())
timestamp = int(datetime.datetime.now().timestamp())
env = RecordVideo(env=env, video_folder="./video", name_prefix=timestamp)
rewards = sarsa_n(
    estimator,
    env=env,
    n=n,
    gamma=gamma,
    episode_cnt=1,
)
env.close()